## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [1]:
import warnings
warnings.filterwarnings('ignore')

import ezomero

# Make CellProfiler run without a GUI
import cellprofiler_core.preferences as cp_preferences
cp_preferences.set_headless()

from cp_omero_toolbox import *

import tempfile


Bad key text.latex.preview in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95    

In [2]:
# Tell CellProfiler to get input from and save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
input_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)
cp_preferences.set_default_image_directory(input_dir.name)

Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [3]:
# Creating a connection object
host = "bioimage.france-bioinformatique.fr"
port = 4074
conn = ezomero.connect(host=host, port=port)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that we can tell our new connection to say "Hi, I'm still here"
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

True

Time to grasp a Dataset from OMERO and download a CellProfiler pipeline that is attached to it. Go to the browser, select a dataset and copy the ID.

In [4]:
dataset_id = int(input("Dataset id: "))


We create a new pipeline with that file, and we remove the first 4 modules. The first 4 modules are in charge of preparing the image data when they are loaded from disk. We don't need them here because we are using OMERO.


In [5]:
measurements_dfs = run_cp_pipeline(conn=conn,
                                   dataset_id=dataset_id,
                                   objects_to_image_table="spots",
                                   objects_to_mask=["nuclei"],
                                   objects_to_point=["spots"],
                                   output_dir=output_dir,
                                   input_dir=input_dir)

Downloaded /tmp/tmpnkvqyy4y/Megane_SpotInNuclei-encours_test_V3.cppipe
Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Pipeline modules:
1 CorrectIlluminationCalculate
2 CorrectIlluminationCalculate
3 CorrectIlluminationApply
4 IdentifyPrimaryObjects
5 IdentifyPrimaryObjects
6 RelateObjects
7 MeasureObjectIntensity
8 RelateObjects
9 ConvertObjectsToImage
10 ConvertObjectsToImage
11 SaveImages
12 SaveImages


We can now start feeding images into the pipeline

In [6]:
cols = measurements_dfs["Image"].columns
vals = [measurements_dfs["Image"][c].values.tolist() for c in cols]

print(len(vals))
print(vals)



48
[[25.0, 25.0, 24.0, 19.0, 12.0], [429.0, 521.0, 365.0, 603.0, 238.0], [694.0, 906.0, 738.0, 1204.0, 270.0], [0.019999999999999574, 0.00999999999999801, 0.0, 0.010000000000005116, 0.0], [0.009999999999999787, 0.010000000000001563, 0.00999999999999801, 0.00999999999999801, 0.0], [1.5199999999999996, 1.4299999999999997, 0.4800000000000004, 0.46999999999999886, 0.47000000000000597], [0.7299999999999995, 0.7399999999999984, 0.4599999999999973, 0.4899999999999949, 0.46999999999999886], [0.05000000000000071, 0.03999999999999915, 0.030000000000001137, 0.04000000000000625, 0.03999999999999915], [3.58, 3.5600000000000023, 3.490000000000002, 3.4399999999999977, 3.519999999999996], [2.2200000000000006, 2.2200000000000024, 2.200000000000003, 2.1799999999999997, 2.1899999999999977], [0.3999999999999986, 0.39000000000000057, 0.4099999999999966, 0.4099999999999966, 0.3900000000000077], [0.2400000000000002, 0.23999999999999488, 0.23000000000000398, 0.23000000000000398, 0.36999999999999744], [0.06000

In [10]:
# remove the output directory
output_dir.cleanup()

# and close the connection to the OMERO server
conn.close()